# Record Linkage com Python e Pandas


O record linkage, também conhecido como resolução de entidade ou vinculação de registros, é uma técnica essencial na integração de bases de dados, especialmente no campo da saúde pública. Seu objetivo é identificar e combinar registros que se referem ao mesmo indivíduo, mesmo que estejam dispersos em sistemas diferentes ou apareçam duplicados em uma única base.


### 🩺 Aplicações na Saúde Pública
Na vigilância em saúde, o linkage permite construir conjuntos de dados mais completos, viabilizando análises mais aprofundadas sobre agravos, doenças e casos específicos. Exemplos:

 *   Relacionar dados do SINASC (nascidos vivos) com o SIM (mortalidade) para estudar mortalidade infantil.

 *   Cruzar o SIVEP-Gripe com o SI-PNI para avaliar a eficácia vacinal contra COVID-19.

### ⚠️ Desafios do Record Linkage
Entre os principais desafios estão:

 *    Erros de digitação

 *   Ausência de identificadores únicos (como CPF)

 *   Inconsistências nos nomes ou datas

Para contornar esses problemas, utilizamos métodos determinísticos, probabilísticos ou híbridos, comparando múltiplos campos com algoritmos específicos (ex: Jaro-Winkler ou Levenshtein).



# 🐍 Implementação em Python com Pandas



No Python, utilizamos a biblioteca pandas para manipulação dos dados e o pacote recordlinkage para aplicar técnicas de linkage. Aqui está um exemplo básico com a abordagem determinística:


In [96]:

pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [97]:
import pandas as pd
from IPython.display import display


# Tabela de resultados

Carrega a função read_csv() do pacote pandas

A função read_csv() é utilizada para ler arquivos CSV em Python.
Ao utilizarmos o parâmetro `sep=';'`, indicamos que o separador de colunas do arquivo é o ponto e vírgula (;)
Esse é o padrão de separação em muitos arquivos CSV gerados por sistemas brasileiros,
como os arquivos exportados de sistemas públicos de saúde

Lê o arquivo chamado "sivep_identificado.csv", que está dentro da pasta "DATA"
Este arquivo contém informações de internações por Síndrome Respiratória Aguda Grave (SRAG), incluindo casos de COVID-19
A base foi extraída do sistema SIVEP-Gripe, utilizado para registrar e acompanhar esses casos no Brasil
O nome do arquivo indica que os dados estão identificados (ou seja, possivelmente contêm nomes ou outros dados sensíveis que permitem identificar os pacientes)

A função read_csv() transforma esse arquivo CSV em um objeto do tipo DataFrame, que será armazenado na variável `sivep`
Essa variável agora contém toda a base de dados de internações e poderá ser manipulada, filtrada,
cruzada com outras bases (como a de óbitos), analisada e visualizada

In [98]:

# Se os campos são separados por ponto e vírgula (padrão do Brasil, como arquivos do SUS):
df = pd.read_csv('/home/pamela/Documentos/Linkage_Data_Health/DATA/sivep_identificado.csv', sep=';')


In [4]:
# Mostra as primeiras linhas
print(df.head(100))

    nu_notific                       nome       sexo   data_nasc  idade  \
0     10001569      Kauan Azevedo Azevedo  masculino  1965-07-27     57   
1     10009876         Raissa Cunha Costa   feminino  1939-07-15     83   
2     10012252      Rafael Castro Barbosa  masculino  1976-03-28     46   
3     10012410     Vinicius Silva Ribeiro  masculino  1964-02-10     58   
4     10017778       Beatriz Araujo Rocha   feminino  1992-03-04     30   
..         ...                        ...        ...         ...    ...   
95    10187056    Vitoria Correia Almeida   feminino  1979-07-25     43   
96    10188551  Carlos Goncalves Carvalho  masculino  1972-12-16     49   
97    10188869       Paulo Barros Almeida  masculino  1972-01-09     50   
98    10189965   Guilherme Castro Barbosa  masculino  1951-12-04     70   
99    10190488       Marisa Ribeiro Pinto   feminino  1961-12-13     60   

               cpf                   nome_mae  
0   622.290.767-95     Analia Azevedo Azevedo  
1  

A visualização abaixo mostra as primeiras linhas do DataFrame `sivep`, que contém informações sobre internações
por Síndrome Respiratória Aguda Grave (SRAG), extraídas do sistema SIVEP-Gripe.
Cada linha representa um paciente e as colunas trazem atributos relacionados à identificação e características individuais.

Explicação das colunas apresentadas:
- nu_notific: número da notificação (identificador único do registro)
- nome: nome completo do paciente
- sexo: sexo biológico (masculino/feminino)
- data_nasc: data de nascimento
- idade: idade do paciente no momento da notificação
- cpf: número do CPF do paciente
- nome_mae: nome completo da mãe do paciente

Essa tabela é útil para realizar análises de perfil demográfico, validar registros por linkage com outras bases (ex: óbitos),
ou realizar estudos epidemiológicos sobre os casos registrados.


# 2. Gerando e filtrando os pares


Realizando a blocagem dos dados pela variável "sexo"
A função 'pair_blocking' do pacote 'reclin' é utilizada para blocar os dados
especificando que a comparação será feita com base na variável "sexo".
Isso ajuda a reduzir a quantidade de pares a serem comparados.
O argumento 'deduplication = TRUE' indica que estamos realizando a deduplicação,
ou seja, comparando o banco de dados consigo mesmo.

In [86]:
# Verificar as colunas para garantir que 'sexo' está presente
print(df.columns)
df=df.head(100)

Index(['nu_notific', 'nome', 'sexo', 'data_nasc', 'idade', 'cpf', 'nome_mae'], dtype='object')


Index(['nu_notific', 'nome', 'sexo', 'data_nasc', 'idade', 'cpf', 'nome_mae'], dtype='object')


In [87]:
# Realizando a blocagem dos dados pela variável "sexo"
pares_blocagem = []

# Agrupar os dados pelo sexo
for sexo, grupo in df.groupby('sexo'):
    # Para cada grupo, realizar a comparação de todos os pares dentro do grupo
    for i in range(len(grupo)):
        for j in range(i + 1, len(grupo)):
            pares_blocagem.append((grupo.iloc[i], grupo.iloc[j]))

# Convertendo a lista de pares em um DataFrame para visualização
pares_blocagem_df = pd.DataFrame(pares_blocagem, columns=['Registro_1', 'Registro_2'])

# Exibindo as primeiras linhas da tabela resultante da blocagem
pares_blocagem_df.head()

# Contando o total de pares encontrados
total_pares = len(pares_blocagem_df)
print(f'Total de pares encontrados: {total_pares}')

Total de pares encontrados: 2454


In [81]:
pip install recordlinkage


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# 3. Aplicando o método de linkage determinístico

### 🔧 import recordlinkage

Esse comando importa o pacote inteiro, permitindo acesso a todos os seus módulos, como:

**recordlinkage.Index()** – para definir blocagens e gerar pares de registros;

**recordlinkage.Compare()** – para comparar os pares gerados e calcular similaridades;

**recordlinkage.ECMClassifier()** – um classificador probabilístico para decidir se pares são correspondências.



### from recordlinkage import Compare
Esse comando importa só a **classe Compare**, usada para:

✅ Definir como os registros serão comparados
Você define as variáveis a comparar, e o método de comparação (ex: Jaro-Winkler, exata, Levenshtein etc).

✅ Calcular a similaridade dos pares


In [88]:
import recordlinkage
from recordlinkage import Compare

Agora aplicaremos o linkage do tipo determinístico. Isto porque com ele o pareamento dos indivíduos no banco de dados é feito pela correspondência exata entre registros. Vamos lá!

Como primeira etapa, iremos realizar o linkage informando quais variáveis queremos comparar. Em nosso exemplo, iremos utilizar as variáveis nome, data de nascimento, cpf e nome da mãe.

Então precisaremos comparar se os valores das variáveis são iguais. Ao aplicar esta etapa poderemos obter os valores TRUE (verdadeiro) ou FALSE (falso). Para que você possa compreender melhor, se um par de registros no qual as quatro variáveis escolhidas fossem TRUE (verdadeiras) representaria um linkage com alta acurácia entre estes registros.

In [92]:

# Criando o indexador e fazendo a blocagem por 'sexo'
indexador = Index()
indexador.block('sexo')

# Gerando os pares de blocagem (isso retorna um pandas.MultiIndex)
pares_blocagem = indexador.index(df)


In [93]:
resultados_comparacao = compare.compute(pares_blocagem, df)


**Compare()**.exact(...): compara os valores de forma exata.

**pares_iguais**: guarda apenas os pares em que todas as variáveis são `iguais (1.0)`.

**block('sexo')**: opcional, usado para reduzir o número de pares comparados, fazendo comparações só dentro de grupos (por exemplo, homens com homens e mulheres com mulheres).

Ao final, o código imprime os pares de registros que batem exatamente em todas as variáveis.



In [90]:

# Suponha que você tenha um DataFrame de exemplo como este:
# O DataFrame 'p_deter' é o resultado da comparação entre os registros, 
# onde as colunas 'nome', 'data_nasc', 'cpf' e 'nome_mae' contêm valores booleanos.
# Aqui, vamos criar um DataFrame fictício para simular o exemplo:

data = {
    'x': [528, 933, 978, 1023],
    'y': [3372, 6984, 3026, 9323],
    'nome': [True, True, True, True],
    'data_nasc': [True, True, True, True],
    'cpf': [True, True, True, True],
    'nome_mae': [True, True, True, True]
}

# Criando o DataFrame
p_deter = pd.DataFrame(data)

# Filtrando os pares com correspondência exata em todas as variáveis
pares_iguais = p_deter[
    p_deter[['nome', 'data_nasc', 'cpf', 'nome_mae']].all(axis=1)
]

# Exibindo os pares com correspondência perfeita
print(pares_iguais)

# A saída será algo assim:
#       x     y   nome  data_nasc   cpf  nome_mae
# 0   528  3372   True   True     True    True
# 1   933  6984   True   True     True    True
# 2   978  3026   True   True     True    True
# 3  1023  9323   True   True     True    True


      x     y  nome  data_nasc   cpf  nome_mae
0   528  3372  True       True  True      True
1   933  6984  True       True  True      True
2   978  3026  True       True  True      True
3  1023  9323  True       True  True      True


 A saída será algo assim:
 | x   |  y  | nome | data_nasc  | cpf |nome_mae|
| ------  | :-----------------------: | ------  | :-----------------------: | ------  | :-----------------------: |
| 0   |528 | 3372  | True   |True |    True|    True|
| 1  | 933 | 6984 |  True |  True |    True|   True|
| 2  | 978 | 3026 |  True  | True |  True |   True|
| 3  |1023 | 9323 |  True  | True  |   True |   True|





Esse resultado mostra os pares de registros que foram identificados como iguais em todas as variáveis comparadas — ou seja, possivelmente registros duplicados no seu dataset.


📋 Descrição do Resultado

| Coluna | Significado |
| ------  | :-----------------------: |
| `x` |	Índice do primeiro registro no par comparado|
| `y`	|  Índice do segundo registro no par comparado |
| `nome` | `True` indica que os valores de nome são iguais nos dois registros |
| `data_nasc`	| `True` indica que as datas de nascimento também batem exatamente |
| `cpf` |	`True` significa que os dois registros têm o mesmo CPF |
| `nome_mae` |	`True` indica que os nomes das mães são iguais |


In [91]:
pip install tabulate


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [99]:
from tabulate import tabulate

In [100]:
# Verifique o número de linhas no DataFrame# Verifique os índices em 'pares_iguais'
print(pares_iguais)


      x     y  nome  data_nasc   cpf  nome_mae
0   528  3372  True       True  True      True
1   933  6984  True       True  True      True
2   978  3026  True       True  True      True
3  1023  9323  True       True  True      True


In [101]:
# Verifique o número de linhas no DataFrame
print(len(df))

10196


In [102]:
# Verifique o número de linhas no DataFrame
print(f"Total de linhas no DataFrame: {len(df)}")

# Suponha que 'pares_iguais' seja um DataFrame com os índices dos pares duplicados
# Exemplo de pares_iguais (índices das linhas duplicadas)
pares_iguais = pd.DataFrame({'x': [528, 1034], 'y': [3372, 2104]})

# Verifique se os índices estão dentro do intervalo
if (pares_iguais['x'][0] < len(df)) and (pares_iguais['y'][0] < len(df)):
    # Selecionando as linhas correspondentes aos índices dos pares duplicados
    registro_1 = df.iloc[pares_iguais['x'][0]]
    registro_2 = df.iloc[pares_iguais['y'][0]]

    # Concatenando as duas linhas para visualização
    pares_duplicados = pd.concat([registro_1, registro_2], axis=1).T

    # Exibindo as duas linhas duplicadas em formato de tabela bonita
    print(tabulate(pares_duplicados, headers='keys', tablefmt='pretty'))
else:
    print("Índices fora do intervalo do DataFrame")

Total de linhas no DataFrame: 10196
+------+------------+-------------------------------+-----------+------------+-------+----------------+-------------------------------+
|      | nu_notific |             nome              |   sexo    | data_nasc  | idade |      cpf       |           nome_mae            |
+------+------------+-------------------------------+-----------+------------+-------+----------------+-------------------------------+
| 528  |  11025479  |    Sophia Araujo Oliveira     | feminino  | 1976-02-04 |  46   | 452.848.419-64 |    Elizia Araujo Oliveira     |
| 3372 |  16522775  | Matheus Cavalcanti Cavalcanti | masculino | 1951-12-22 |  70   | 850.491.708-85 | Deborah Cavalcanti Cavalcanti |
+------+------------+-------------------------------+-----------+------------+-------+----------------+-------------------------------+



Total de linhas no DataFrame: 10196

| nu_notific |             nome              |   sexo    | data_nasc  | idade |      cpf       |           nome_mae            |
| ------  | :-----------------------: | ------  | :-----------------------: | ------  |------  | :-----------------------: |
| 528  |  11025479  |    Sophia Araujo Oliveira     | feminino  | 1976-02-04 |  46   | 452.848.419-64 |    Elizia Araujo Oliveira     |
| 3372 |  16522775  | Matheus Cavalcanti Cavalcanti | masculino | 1951-12-22 |  70   | 850.491.708-85 | Deborah Cavalcanti Cavalcanti |


In [115]:
pip install recordlinkage



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [109]:

# Verifique o número de linhas no DataFrame
total_linhas = len(df)
print(f"Total de linhas no DataFrame: {total_linhas}")

# Suponha que 'pares_iguais' seja um DataFrame com os índices dos pares duplicados
# Exemplo de pares_iguais (índices das linhas duplicadas)
pares_iguais = pd.DataFrame({'x': [528, 1034], 'y': [3372, 2104]})

# Verifique os índices em 'pares_iguais'
print("Índices em pares_iguais:")
print(pares_iguais)

# Verifique se os índices estão dentro do intervalo
for index_x, index_y in zip(pares_iguais['x'], pares_iguais['y']):
    if index_x < total_linhas and index_y < total_linhas:
        # Selecionando as linhas correspondentes aos índices dos pares duplicados
        registro_1 = df.iloc[index_x]
        registro_2 = df.iloc[index_y]

        # Concatenando as duas linhas para visualização
        pares_duplicados = pd.concat([registro_1, registro_2], axis=1).T

        # Exibindo as duas linhas duplicadas em formato de tabela bonita
        print(tabulate(pares_duplicados, headers='keys', tablefmt='pipe', showindex=False))
    else:
        print(f"Índices {index_x} ou {index_y} estão fora do intervalo do DataFrame")


Total de linhas no DataFrame: 10196
Índices em pares_iguais:
      x     y
0   528  3372
1  1034  2104
|   nu_notific | nome                          | sexo      | data_nasc   |   idade | cpf            | nome_mae                      |
|-------------:|:------------------------------|:----------|:------------|--------:|:---------------|:------------------------------|
|  1.10255e+07 | Sophia Araujo Oliveira        | feminino  | 1976-02-04  |      46 | 452.848.419-64 | Elizia Araujo Oliveira        |
|  1.65228e+07 | Matheus Cavalcanti Cavalcanti | masculino | 1951-12-22  |      70 | 850.491.708-85 | Deborah Cavalcanti Cavalcanti |
|   nu_notific | nome                       | sexo     | data_nasc   |   idade | cpf            | nome_mae                  |
|-------------:|:---------------------------|:---------|:------------|--------:|:---------------|:--------------------------|
|  1.20815e+07 | Isabela Costa Araujo       | feminino | 1955-03-30  |      67 | 658.203.238-07 | Mauricia Cl

In [116]:
import recordlinkage
from recordlinkage import compare
from recordlinkage.index import Block


In [117]:

# Supondo que você tenha um DataFrame 'df' com as colunas 'nome', 'data_nasc', 'cpf', 'nome_mae'

# Limpeza de dados, como remoção de espaços extras e conversão de maiúsculas/minúsculas
df['nome'] = clean(df['nome'])
df['cpf'] = clean(df['cpf'])
df['nome_mae'] = clean(df['nome_mae'])
df['data_nasc'] = clean(df['data_nasc'])

# Criação do índice de comparação
indexer = Index()
indexer.block('nome')  # Comparando primeiro pelo nome
pairs = indexer.index(df)

# Comparação de pares
compare = recordlinkage.Compare()

# Adicionando comparações para as colunas
compare.string('nome', 'nome', threshold=0.9, label='nome')
compare.string('cpf', 'cpf', threshold=0.9, label='cpf')
compare.string('nome_mae', 'nome_mae', threshold=0.9, label='nome_mae')
compare.string('data_nasc', 'data_nasc', threshold=0.9, label='data_nasc')

# Comparando os pares
features = compare.compute(pairs, df)

# Resultados: Filtrando os pares com alta similaridade
matches = features[features.sum(axis=1) > 2]  # Ajuste o número conforme necessário

# Exibindo os pares com alta similaridade
if not matches.empty:
    for index_x, index_y in matches.index:
        registro_1 = df.iloc[index_x]
        registro_2 = df.iloc[index_y]
        pares_duplicados = pd.concat([registro_1, registro_2], axis=1).T
        print(pares_duplicados.to_string(index=False))
else:
    print("Nenhum par duplicado encontrado.")


nu_notific                 nome     sexo  data_nasc idade          cpf                 nome_mae
  11202707 livia oliveira alves feminino 1962 12 25    59 773766365 79 carolaine oliveira alves
  10709881 livia oliveira alves feminino 1962 12 25    59 773766365 79 karolaine oliveira alves
nu_notific             nome      sexo  data_nasc idade          cpf           nome_mae
  12407867 joao cunha pinto masculino 1970 06 19    52 304483301 99 helida cunha pinto
  11794257 joao cunha pinto masculino 1970 06 19    52 304483300 99 merida cunha pinto
nu_notific                  nome      sexo  data_nasc idade          cpf             nome_mae
  12884905 vincius correia costa masculino 1983 01 18    39 376049654 74 camila correia costa
  10072562 vincius correia costa masculino 1983 01 28    39 377049654 74 camila correia costa
nu_notific                     nome      sexo  data_nasc idade          cpf                      nome_mae
  13530813 fbio fernandes goncalves masculino 1997 01 13    25 

In [118]:
# import pandas as pd
# import recordlinkage
# from recordlinkage import Index
# from recordlinkage.preprocessing import clean
# from recordlinkage import Compare

# Supondo que você tenha um DataFrame 'df' com as colunas 'nome', 'data_nasc', 'cpf', 'nome_mae'

# Limpeza de dados, como remoção de espaços extras e conversão de maiúsculas/minúsculas
df['nome'] = clean(df['nome'])
df['cpf'] = clean(df['cpf'])
df['nome_mae'] = clean(df['nome_mae'])
df['data_nasc'] = clean(df['data_nasc'])

# Verificando duplicatas diretamente
print(df[df.duplicated(subset=['nome', 'cpf', 'nome_mae', 'data_nasc'], keep=False)])

# Criação do índice de comparação
indexer = Index()
indexer.block(['nome', 'data_nasc'])  # Alterando a estratégia de bloqueio
pairs = indexer.index(df)

# Comparação de pares
compare = Compare()

# Ajustando os limiares de similaridade para cada coluna usando o método 'string' com 'jaro_winkler'
compare.string('nome', 'nome', threshold=0.7, label='nome', method='jaro_winkler')
compare.string('cpf', 'cpf', threshold=0.8, label='cpf', method='jaro_winkler')
compare.string('nome_mae', 'nome_mae', threshold=0.8, label='nome_mae', method='jaro_winkler')
compare.string('data_nasc', 'data_nasc', threshold=0.6, label='data_nasc', method='jaro_winkler')

# Comparando os pares
features = compare.compute(pairs, df)

# Exibindo as comparações para diagnóstico
print(features.head())

# Resultados: Filtrando os pares com alta similaridade
matches = features[features.sum(axis=1) > 0]  # Relaxando a condição para permitir mais matches

# Exibindo os pares com alta similaridade
if not matches.empty:
    for index_x, index_y in matches.index:
        registro_1 = df.iloc[index_x]
        registro_2 = df.iloc[index_y]
        pares_duplicados = pd.concat([registro_1, registro_2], axis=1).T
        print(pares_duplicados.to_string(index=False))
else:
    print("Nenhum par duplicado encontrado.")


      nu_notific                       nome       sexo   data_nasc  idade  \
527     11024657     miguel azevedo almeida  masculino  1965 05 24     37   
932     11885430       isabella castro melo   feminino  1991 08 26     30   
977     11975810       luiza oliveira silva   feminino  1988 01 11     64   
1022    12067118        toms ribeiro santos  masculino  1982 11 21     46   
1828    13526107   fbio fernandes goncalves  masculino  1997 01 13     25   
1829    13530813   fbio fernandes goncalves  masculino  1997 01 13     25   
2128    14076116           vitor lima gomes  masculino  1968 12 12     53   
2222    14229552     rebeca azevedo martins   feminino  1975 12 19     46   
2549    14850623      alex correia carvalho  masculino  2001 01 04     31   
3025    15836426       luiza oliveira silva   feminino  1988 01 11     34   
3371    16521780     miguel azevedo almeida  masculino  1965 05 24     57   
3545    16849021      alex rocha cavalcanti  masculino  1976 11 05     45   

In [ ]:

# Supondo que você tenha um DataFrame 'df' com as colunas 'nome', 'data_nasc', 'cpf', 'nome_mae'

# Limpeza de dados, como remoção de espaços extras e conversão de maiúsculas/minúsculas
df['nome'] = clean(df['nome'])
df['cpf'] = clean(df['cpf'])
df['nome_mae'] = clean(df['nome_mae'])
df['data_nasc'] = clean(df['data_nasc'])

# Verificando duplicatas diretamente
print("Verificando duplicatas diretamente:")
print(df[df.duplicated(subset=['nome', 'cpf', 'nome_mae', 'data_nasc'], keep=False)])

# Criando o índice de comparação
indexer = Index()
indexer.block(['sexo'])  # Alterando a estratégia de bloqueio para 'sexo' (ajuste conforme necessário)
pairs = indexer.index(df)

# Exibindo o número total de pares
print(f"\nNúmero total de pares: {len(pairs)} pares")

# Comparação de pares
compare = Compare()

# Ajustando os limiares de similaridade para cada coluna usando o método 'string' com 'jaro_winkler'
compare.string('nome', 'nome', threshold=0.7, label='nome', method='jaro_winkler')
compare.string('cpf', 'cpf', threshold=0.8, label='cpf', method='jaro_winkler')
compare.string('nome_mae', 'nome_mae', threshold=0.8, label='nome_mae', method='jaro_winkler')
compare.string('data_nasc', 'data_nasc', threshold=0.6, label='data_nasc', method='jaro_winkler')

# Comparando os pares
features = compare.compute(pairs, df)

# Exibindo o resumo de similaridade
print("\nResumo da similaridade:")
print(features.head())

# Filtrando os pares com alta similaridade
matches = features[features.sum(axis=1) > 0]  # Relaxando a condição para permitir mais matches

# Exibindo os pares com alta similaridade
if not matches.empty:
    print("\nPares duplicados encontrados:")
    for index_x, index_y in matches.index:
        registro_1 = df.iloc[index_x]
        registro_2 = df.iloc[index_y]
        pares_duplicados = pd.concat([registro_1, registro_2], axis=1).T
        print(pares_duplicados.to_string(index=False))
else:
    print("\nNenhum par duplicado encontrado.")


In [113]:

# Limpeza básica
df['nome'] = clean(df['nome'])
df['cpf'] = clean(df['cpf'])
df['nome_mae'] = clean(df['nome_mae'])
df['data_nasc'] = clean(df['data_nasc'])

# Número de registros
total_linhas = len(df)
print(f"# Base de dados: {total_linhas} registros")

# Criando pares com bloqueio pela variável 'sexo'
indexer = Index()
indexer.block('sexo')
pares = indexer.index(df)
print(f"# Número total de pares: {len(pares):,} pares")
print("# Bloqueando pela variável: 'sexo'\n")

# Comparador
compare = Compare()
compare.string('nome', 'nome', method='jaro_winkler', threshold=0.7, label='nome')
compare.string('cpf', 'cpf', method='jaro_winkler', threshold=0.8, label='cpf')
compare.string('nome_mae', 'nome_mae', method='jaro_winkler', threshold=0.8, label='nome_mae')
compare.string('data_nasc', 'data_nasc', method='jaro_winkler', threshold=0.6, label='data_nasc')

# Computando as similaridades
features = compare.compute(pares, df)

# Formatando os números com 7 casas decimais
pd.set_option("display.float_format", lambda x: f"{x:.7f}")

# Adicionando colunas dos índices .x e .y para facilitar a visualização
features_reset = features.reset_index()

# Exibindo os 5 primeiros pares comparados com valores formatados
print("#       .x   .y     nome      data_nasc   cpf       nome_mae")
for idx, row in features_reset.head(5).iterrows():
    print(f"# {idx+1: <6} {int(row['level_0']): <4} {int(row['level_1']): <4} "
          f"{row['nome']: <10.7f} {row['data_nasc']: <10.7f} {row['cpf']: <10.7f} {row['nome_mae']: <10.7f}")


# Base de dados: 10196 registros
# Número total de pares: 25,985,290 pares
# Bloqueando pela variável: 'sexo'



KeyboardInterrupt: 

In [ ]:
pd.set_option('display.float_format', '{:.6f}'.format)

In [ ]:
# import pandas as pd
# import recordlinkage
# from recordlinkage import Index, Compare
# from recordlinkage.preprocessing import clean

# Pré-processamento
df['nome'] = clean(df['nome'])
df['cpf'] = clean(df['cpf'])
df['nome_mae'] = clean(df['nome_mae'])
df['data_nasc'] = clean(df['data_nasc'])

# Número de registros
total_linhas = len(df)
print(f"# Base de dados: {total_linhas} registros")

# Indexação por 'sexo'
indexer = Index()
indexer.block('sexo')
pares = indexer.index(df)
print(f"# Número total de pares: {len(pares):,} pares")
print("# Bloqueando pela variável: 'sexo'\n")

# Comparação
compare = Compare()
compare.string('nome', 'nome', method='jaro_winkler', threshold=0.7, label='nome')
compare.string('cpf', 'cpf', method='jaro_winkler', threshold=0.8, label='cpf')
compare.string('nome_mae', 'nome_mae', method='jaro_winkler', threshold=0.8, label='nome_mae')
compare.string('data_nasc', 'data_nasc', method='jaro_winkler', threshold=0.6, label='data_nasc')

# Executando comparação
features = compare.compute(pares, df)

# Convertendo para DataFrame e resetando índices
resultados = features.reset_index()

# Cabeçalho com tipo de dados estilo R
print("#       .x   .y     nome   data_nasc   cpf     nome_mae")
print("#    <int> <int>   <num>   <num>       <num>   <num>")

# Exibindo os primeiros pares no formato desejado
for i, row in resultados.head(10).iterrows():  # Altere o número conforme quiser
    print(f"{i+1}: {row['level_0']} {row['level_1']}  "
          f"{row['nome']}  {row['data_nasc']}  "
          f"{row['cpf']:>9.7f}  {row['nome_mae']:>9.7f}")
